# Titan Crypto - Iceberg Debug Notebook
This notebook initializes Spark with all necessary Iceberg and MinIO configurations to debug data loading and table creation.

In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# 1. Initialize Spark with Iceberg + MinIO Config
spark = SparkSession.builder \
    .appName("Titan-Jupyter-Debug") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.host", "titan-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.iceberg:iceberg-aws-bundle:1.4.2") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.titan", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.titan.type", "rest") \
    .config("spark.sql.catalog.titan.uri", "http://iceberg-catalog:8181") \
    .config("spark.sql.catalog.titan.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.titan.warehouse", "s3://warehouse") \
    .config("spark.sql.catalog.titan.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.titan.s3.access-key-id", "admin") \
    .config("spark.sql.catalog.titan.s3.secret-access-key", "password") \
    .config("spark.sql.catalog.titan.s3.path-style-access", "true") \
    .config("spark.sql.catalog.titan.client.region", "us-east-1") \
    .config("spark.sql.catalog.titan.s3.ssl.enabled", "false") \
    .getOrCreate()

print("Spark Session initialized with Iceberg support.")

In [ ]:
# 2. Verify Iceberg Catalog
spark.sql("SHOW NAMESPACES IN titan").show()

In [ ]:
# 3. Test Create Table (Same as main.py)
spark.sql("CREATE NAMESPACE IF NOT EXISTS titan.crypto")
spark.sql("""
    CREATE TABLE IF NOT EXISTS titan.crypto.btc_trades (
        open double, high double, low double, close double, volume double,
        quote_asset_volume double, trades long, taker_buy_base double,
        taker_buy_quote double, event_time timestamp
    ) USING iceberg PARTITIONED BY (months(event_time))
""")
print("Table created/verified.")